In [ ]:
import pandas as pd
import pathlib
from addict import Dict
import yaml
from stability.data import CellDataset
import numpy as np
import stability.features as sf
from stability.models.vae import VAE
import stability.train as st
import torch
import torch.optim
from torch.utils.data import DataLoader

In [ ]:
#data_dir = pathlib.Path(os.environ["DATA_DIR"])
# root_dir = pathlib.Path(os.environ["ROOT_DIR"])
data_dir = pathlib.Path("/Users/kris/Documents/stability_data/")
root_dir = pathlib.Path("/Users/kris/Desktop/conceptual/learned_inference/")
opts = Dict(yaml.safe_load(open(root_dir / "conf/train.yaml", "r")))


In [ ]:
splits = pd.read_csv(data_dir / opts.organization.splits)
resample_ix = pd.read_csv(data_dir / opts.bootstrap.path)
train_paths = splits.loc[splits.split == "train", "path"].values

In [ ]:
# Setup model
model = VAE(z_dim=opts.train.z_dim)
optim = torch.optim.Adam(model.parameters(), lr=opts.train.lr)

# train
# you're going to need to match the y-value... probably read the original and match
cell_boot = CellDataset(train_paths[resample_ix.loc[0]], data_dir / opts.organization.xy)
cell_ft = CellDataset(train_paths, data_dir / opts.organization.xy)

train_loader = DataLoader(cell_boot, batch_size=opts.train.batch_size, shuffle=True)
ft_loader = DataLoader(cell_ft, batch_size=opts.train.batch_size, shuffle=False)

h = sf.save_features(ft_loader, model, 0, "tmp")
#np.save("/Users/kris/Desktop/test.npy", h["layer_1"].numpy())